# Moonshot 5 Hackathon - Drought in the Netherlands
Using RA2CE to assess the capacity of the road network between Rotterdam and the Ruhr area.

In [7]:
from ra2ce.ra2ce_handler import Ra2ceHandler # import the ra2cehandler to run ra2ce analyses
from pathlib import Path

_network_ini_name = "network.ini" # set the name for the network.ini settings file

folder_dir = Path(r'C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5')

network_ini = folder_dir / _network_ini_name
assert network_ini.is_file() # check whether the network.ini file exists

In [8]:
race = Ra2ceHandler(network=network_ini, analysis=None)
race.configure()

2023-09-15 02:19:41 PM - [ra2ce_logging.py:41] - root - INFO - RA2CE logger initialized.
2023-09-15 02:19:41 PM - [ra2ce_logging.py:41] - root - INFO - RA2CE logger initialized.
2023-09-15 02:19:41 PM - [ra2ce_logging.py:41] - root - INFO - RA2CE logger initialized.
2023-09-15 02:19:41 PM - [network_wrapper_factory.py:53] - root - INFO - Start creating a network from the submitted shapefile.
2023-09-15 02:19:41 PM - [network_wrapper_factory.py:53] - root - INFO - Start creating a network from the submitted shapefile.
2023-09-15 02:19:41 PM - [network_wrapper_factory.py:53] - root - INFO - Start creating a network from the submitted shapefile.
2023-09-15 02:19:41 PM - [osm_network_wrapper.py:55] - root - INFO - Start downloading a network from OSM.
2023-09-15 02:19:41 PM - [osm_network_wrapper.py:55] - root - INFO - Start downloading a network from OSM.
2023-09-15 02:19:41 PM - [osm_network_wrapper.py:55] - root - INFO - Start downloading a network from OSM.
